## Taller 02: Modelo de Espacio Vectorial

**Objetivo:**  
Comprender y aplicar el Modelo de Espacio Vectorial para el ranking de documentos basado en la similitud de contenido con una consulta dada.

**Parte 1: Preparación de Datos**

**Creación de la Matriz de Términos-Documento**

Cada grupo de estudiantes deberá crear una tabla con las categorías de palabras como filas y los textos como columnas. Para cada texto, los estudiantes contarán cuántas veces aparecen palabras pertenecientes a cada categoría y llenarán la tabla con estos conteos.


In [1]:
import re
import pandas as pd
from unicodedata import normalize

In [2]:
def limpiar_string(texto):
    texto = texto.lower()
    texto = texto.replace("ñ", "n")
    texto = re.sub(
        r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1",
        normalize("NFD", texto), 0, re.I
    )
    texto = re.sub(r'[^a-zA-Z\s]', '', texto)
    terminos = texto.split()
    return terminos

In [3]:
textos = [
    "Historia de Madrid: Madrid, la capital de España, tiene una rica historia que se remonta al siglo IX. La ciudad fue fundada por los musulmanes, y su nombre proviene del árabe “Majrit” que significa “fuente de agua”. A lo largo de los siglos, Madrid se ha convertido en un centro político, económico y cultural de España.",
    "El Palacio Real: El Palacio Real de Madrid es una de las residencias oficiales de la familia real española, aunque ahora se usa principalmente para ceremonias de estado. Con más de 3,000 habitaciones, es el palacio real más grande de Europa Occidental y un testimonio del esplendor y la historia de España.",
    "Museo del Prado: El Museo del Prado es uno de los museos más famosos del mundo y un destino imprescindible para los amantes del arte que visitan Madrid. Alberga una impresionante colección de obras de Velázquez, Goya, El Greco y otros maestros europeos.",
    "Parque del Retiro: El Parque del Retiro es el pulmón verde de Madrid. Antiguamente perteneciente a la monarquía española, este parque de 118 hectáreas ofrece un respiro de la vida urbana con sus estanques, jardines y el Palacio de Cristal.",
    "Gastronomía Madrileña: Madrid es famosa por su vibrante escena gastronómica que incluye desde tapas en pequeños bares hasta restaurantes de alta cocina. Platos típicos como el cocido madrileño y las patatas bravas son esenciales para cualquier visitante.",
    "Gran Vía: La Gran Vía es una de las calles más famosas de Madrid, conocida por sus tiendas, teatros y arquitectura emblemática. Este bullicioso bulevar es un punto de encuentro para turistas y locales por igual.",
    "Real Madrid y el Estadio Santiago Bernabéu: El Real Madrid, uno de los clubes de fútbol más famosos y exitosos del mundo, juega sus partidos en el Estadio Santiago Bernabéu. Este estadio es una meca para los aficionados al fútbol y ofrece tours que permiten a los visitantes explorar su rica historia.",
    "Navidad en Madrid: Durante la temporada navideña, Madrid se transforma con mercados festivos, luces deslumbrantes y decoraciones. La Plaza Mayor se convierte en un enorme mercado de Navidad, donde se pueden encontrar todo tipo de regalos y delicias tradicionales.",
    "Nochevieja en Puerta del Sol: La Puerta del Sol es el corazón de Madrid y el lugar donde miles se reúnen cada Nochevieja para dar la bienvenida al año nuevo. Comer las doce uvas de la suerte al son de las campanadas del reloj es una tradición que todos deberían experimentar.",
    "Madrid Río: Madrid Río es un parque lineal a lo largo del río Manzanares y un ejemplo perfecto de cómo la ciudad ha transformado áreas industriales en espacios verdes vibrantes. Es un lugar popular para caminar, andar en bicicleta y disfrutar de actividades al aire libre.",
    "Historia de Dortmund: Dortmund, situada en la región del Ruhr en Alemania, tiene una historia que se remonta al año 882. Originalmente una pequeña villa, Dortmund creció para convertirse en una importante ciudad industrial, especialmente conocida por su producción de acero y cerveza.",
    "Westfalenpark: Westfalenpark es uno de los parques urbanos más grandes de Europa y un lugar destacado para los visitantes en Dortmund. El parque es famoso por su Rosarium, que alberga una de las colecciones más extensas de rosas en el mundo, y la torre Florian, que ofrece vistas panorámicas de la ciudad.",
    "Museo del Fútbol Alemán: Inaugurado en 2015, el Museo del Fútbol Alemán ofrece una experiencia interactiva que celebra la rica historia del fútbol alemán. Ubicado en Dortmund, el museo atrae a aficionados de todo el mundo y destaca por sus exposiciones que cuentan desde los primeros días del deporte hasta los triunfos en Copas del Mundo.",
    "Mercado de Navidad de Dortmund: El Mercado de Navidad de Dortmund es uno de los más grandes y más visitados de Alemania. Con su gigantesco árbol de Navidad, posiblemente el más alto de los mercados navideños del país, y más de 300 puestos que venden artesanías, alimentos y bebidas tradicionales, es un evento imperdible durante la temporada festiva.",
    "U-Tower - Centro de Arte y Creatividad: El U-Tower, una antigua cervecería reconvertida, es ahora un centro dinámico de arte y creatividad. Alberga el Museo Ostwall, espacios de exposiciones y estudios de artistas, y es un símbolo de la transformación cultural de Dortmund.",
    "Borussia Dortmund y el Signal Iduna Park: Borussia Dortmund es uno de los clubes de fútbol más populares y exitosos de Alemania. El equipo juega sus partidos en casa en el Signal Iduna Park, conocido por su increíble atmósfera y el famoso “Muro Amarillo”, la tribuna sur que alberga a 25,000 aficionados apasionados.",
    "La cervecería Dortmund: Dortmund tiene una larga tradición cervecera, y la visita a una de sus muchas cervecerías es una parada obligatoria. Las cervecerías ofrecen tours que incluyen degustaciones de cervezas locales como Dortmunder Export, una lager que es un orgullo de la ciudad.",
    "La Reinoldikirche: La iglesia de San Reinold es el edificio más antiguo de Dortmund, datando del siglo XIII. Es un punto de referencia histórico importante y ofrece a los visitantes una mirada al pasado medieval de la ciudad.",
    "Dortmund durante la Segunda Guerra Mundial: Dortmund fue un objetivo significativo durante los bombardeos de la Segunda Guerra Mundial debido a su industria pesada. Hoy en día, varios monumentos y museos ofrecen reflexiones sobre este período y sus efectos en la ciudad.",
    "Zoológico de Dortmund: El Zoológico de Dortmund es un lugar excelente para familias y amantes de la naturaleza. Con más de 1,500 animales y un enfoque especial en la conservación de especies amenazadas, el zoológico proporciona una experiencia educativa y entretenida."
]


In [4]:

textos_limpios = [limpiar_string(texto) for texto in textos]


terminos_unicos = set()
for texto in textos_limpios:
    terminos_unicos.update(texto)

terminos_unicos=sorted(terminos_unicos)
df = pd.DataFrame(0, index=range(len(textos)), columns=terminos_unicos)

for i, texto in enumerate(textos_limpios):
    for termino in texto:
        df.at[i, termino] += 1

df


,a,acero,actividades,aficionados,agua,ahora,aire,al,alberga,aleman,...,visita,visitados,visitan,visitante,visitantes,vistas,westfalenpark,xiii,y,zoologico
0,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,2,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,2,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0
6,1,0,0,1,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,3,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0
8,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,1,0
9,1,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,2,0


In [5]:
import os
if not os.path.exists("archivos_csv_taller"):
    os.makedirs("archivos_csv_taller")
    print(f"Se ha creado la carpeta '{"archivos_csv_taller"}' en el directorio alctual.")

df.to_csv('archivos_csv_taller/docs_y_terminos.csv', index=False)

**Vectorización de Consultas**

Los estudiantes deberán convertir la consulta en un vector, utilizando las mismas cate-
gorías de palabras. El vector podrá contener pesos basados en la importancia percibida de
cada categoría para la consulta

In [6]:
querys = [
    "Historia medieval de las ciudades europeas",
    "Principales destinos turísticos en Europa",
    "Influencia de la realeza en la cultura europea",
    "Importancia de los parques urbanos en las ciudades",
    "Gastronomía típica en capitales europeas",
    "Eventos deportivos icónicos en Europa",
    "Celebraciones de Navidad en ciudades europeas",
    "Museos de arte importantes en Europa",
    "Efectos de la Segunda Guerra Mundial en ciudades europeas",
    "Arquitectura histórica en ciudades europeas",
    "Clubes de fútbol famosos y sus estadios en Europa",
    "Transformación urbana y regeneración de espacios",
    "Tradición cervecera en ciudades europeas",
    "Mercados y comercio tradicional en Europa",
    "Centros de arte y creatividad en ciudades modernas",
    "Actividades de ocio y entretenimiento en ciudades metropolitanas",
    "Conservación de la naturaleza y la vida silvestre en zonas urbanas",
    "Planificación de eventos culturales y festivales en ciudades",
    "Desarrollo del transporte y la infraestructura urbana",
    "Impacto de la tecnología en la vida urbana"
]

df2 = pd.DataFrame({'query': querys, 'ranked': [None] * len(textos)})
df2

,query,ranked
0,Historia medieval de las ciudades europeas,None
1,Principales destinos turísticos en Europa,None
2,Influencia de la realeza en la cultura europea,None
3,Importancia de los parques urbanos en las ciud...,None
4,Gastronomía típica en capitales europeas,None
5,Eventos deportivos icónicos en Europa,None
6,Celebraciones de Navidad en ciudades europeas,None
7,Museos de arte importantes en Europa,None
8,Efectos de la Segunda Guerra Mundial en ciudad...,None
9,Arquitectura histórica en ciudades europeas,None


El anterior dataframe representa las querys en una columna, y el ranked de los 3 mejores textos, por el momento está vacio, pero una vez realicemos el calculo ya no lo estará. A continuación vamos a crear el dataframe que representará las querys en forma de vector. 

In [7]:
df_query = pd.DataFrame(columns=terminos_unicos)
def agregar_query(valores,index):
  df_query.loc[index] = [0] * len(terminos_unicos)
  for texto in valores:
      if texto in df_query.columns:
          df_query.at[index, texto] += 1

In [8]:
querys= df2.iloc[:, 0]
for i,valor in enumerate(querys):
    valor_limpio = limpiar_string(valor)
    agregar_query(valor_limpio,i)
df_query

,a,acero,actividades,aficionados,agua,ahora,aire,al,alberga,aleman,...,visita,visitados,visitan,visitante,visitantes,vistas,westfalenpark,xiii,y,zoologico
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
df.to_csv('archivos_csv_taller/querys_terminos.csv', index=False)

## Cálculo de la Distancia Coseno

Utilizando la matriz de términos-documento y el vector de consulta, cada grupo de estudiantes calculará la distancia coseno entre la consulta y cada texto. La distancia coseno se calcula para determinar cuán similar es cada texto a la consulta en términos de la distribución de categorías de palabras.

## Ranking de Documentos

Basándose en los valores obtenidos de la distancia coseno, los estudiantes ordenarán los textos desde el más relevante al menos relevante respecto a la consulta.


In [10]:
import numpy as np
def calculo_distancia(np_array_query, np_array_text,scores):
  producto_punto = np.dot(np_array_query, np_array_text)

  norma_a = np.linalg.norm(np_array_query)
  norma_b = np.linalg.norm(np_array_text)

  distancia_coseno = producto_punto / (norma_a * norma_b)
  scores.append(distancia_coseno)

In [11]:
titulos=[]
for texto in textos:
    partes = texto.split(":")
    titulo = partes[0].strip()
    titulos.append(titulo)

print(titulos)

['Historia de Madrid', 'El Palacio Real', 'Museo del Prado', 'Parque del Retiro', 'Gastronomía Madrileña', 'Gran Vía', 'Real Madrid y el Estadio Santiago Bernabéu', 'Navidad en Madrid', 'Nochevieja en Puerta del Sol', 'Madrid Río', 'Historia de Dortmund', 'Westfalenpark', 'Museo del Fútbol Alemán', 'Mercado de Navidad de Dortmund', 'U-Tower - Centro de Arte y Creatividad', 'Borussia Dortmund y el Signal Iduna Park', 'La cervecería Dortmund', 'La Reinoldikirche', 'Dortmund durante la Segunda Guerra Mundial', 'Zoológico de Dortmund']


In [12]:
def ranked_skores(scores):
    score_sorted=sorted(scores)
    #best_scores=score_sorted[:3]
    best_titles = []
    titulos_aux=titulos.copy()
    contador=0
    for score in score_sorted:
        if score==0.0:
            print("No hay similitud")
        else:         
            indice = scores.index(score) 
            best_titles.append(titulos[indice])     
            scores[indice]=None
            titulos_aux[indice]=None    
    print("los tres titulos: ", best_titles)
    print("****************************")
    print("\n\n")
    return best_titles

In [13]:
for indice, fila in df_query.iterrows():
    array_query = np.array(fila)
    query=df2.iloc[indice,0]
    print("Query a procesar: ",query)
    scores=[]
    for ind,fil in df.iterrows():
        array_doc=np.array(fil)
        calculo_distancia(array_query,array_doc,scores)
    best_titles=ranked_skores(scores)
    df2.at[indice,'ranked']=best_titles


Query a procesar:  Historia medieval de las ciudades europeas
los tres titulos:  ['Dortmund durante la Segunda Guerra Mundial', 'Museo del Fútbol Alemán', 'Madrid Río', 'Navidad en Madrid', 'Real Madrid y el Estadio Santiago Bernabéu', 'Borussia Dortmund y el Signal Iduna Park', 'Gastronomía Madrileña', 'Museo del Prado', 'La cervecería Dortmund', 'Parque del Retiro', 'Historia de Dortmund', 'Nochevieja en Puerta del Sol', 'Gran Vía', 'Zoológico de Dortmund', 'U-Tower - Centro de Arte y Creatividad', 'Westfalenpark', 'La Reinoldikirche', 'Mercado de Navidad de Dortmund', 'Historia de Madrid', 'El Palacio Real']
****************************



Query a procesar:  Principales destinos turísticos en Europa
No hay similitud
No hay similitud
No hay similitud
No hay similitud
No hay similitud
No hay similitud
No hay similitud
los tres titulos:  ['El Palacio Real', 'Historia de Madrid', 'Real Madrid y el Estadio Santiago Bernabéu', 'Nochevieja en Puerta del Sol', 'Zoológico de Dortmund', 'Gast

# Presentación de Resultados

En nuestra investigación, utilizamos la medida de distancia del coseno para comparar las similitudes entre vectores representativos de consultas y documentos. A continuación, presentamos los resultados obtenidos:

1. **Selección de los mejores resultados:** Utilizando la distancia del coseno, identificamos los tres documentos que presentan la mayor similitud con respecto a una consulta dada. Esta similitud se expresa en forma de puntajes, donde valores cercanos a 0 indican una mayor similitud.

2. **Interpretación de los puntajes:** Los puntajes obtenidos representan la cercanía de los documentos seleccionados con la consulta. Cuanto más cercano a 0 sea el puntaje, mayor será la similitud entre el documento y la consulta. Por lo tanto, los tres mejores resultados son aquellos con los puntajes más bajos.

3. **Implicaciones de los resultados:** Los documentos seleccionados representan las respuestas más relevantes a la consulta dada, según nuestro análisis basado en la distancia del coseno. Estos resultados pueden ser de gran utilidad para entender mejor el contenido relacionado con la consulta y tomar decisiones informadas.

En resumen, nuestra metodología de selección de resultados basada en la distancia del coseno nos permite identificar y presentar de manera efectiva los documentos más relevantes para una consulta dada, teniendo en cuenta que hemos ignorado los puntajes de 0, los cuales indican una falta de similitud entre la consulta y los documentos.


In [14]:
df2

,query,ranked
0,Historia medieval de las ciudades europeas,"[Dortmund durante la Segunda Guerra Mundial, M..."
1,Principales destinos turísticos en Europa,"[El Palacio Real, Historia de Madrid, Real Mad..."
2,Influencia de la realeza en la cultura europea,"[Real Madrid y el Estadio Santiago Bernabéu, G..."
3,Importancia de los parques urbanos en las ciud...,"[La cervecería Dortmund, Parque del Retiro, Ma..."
4,Gastronomía típica en capitales europeas,"[Historia de Madrid, Real Madrid y el Estadio ..."
5,Eventos deportivos icónicos en Europa,"[El Palacio Real, Historia de Madrid, Real Mad..."
6,Celebraciones de Navidad en ciudades europeas,"[Real Madrid y el Estadio Santiago Bernabéu, G..."
7,Museos de arte importantes en Europa,"[Real Madrid y el Estadio Santiago Bernabéu, G..."
8,Efectos de la Segunda Guerra Mundial en ciudad...,"[Real Madrid y el Estadio Santiago Bernabéu, G..."
9,Arquitectura histórica en ciudades europeas,"[Historia de Madrid, Real Madrid y el Estadio ..."


In [15]:
df.to_csv('archivos_csv_taller/resultados_ranked.csv', index=False)

Los archivos .csv están en la ruta Homeworks/week01/notebooks/archivos_csv_taller